# ABS Quarterly Producer Prices Index 6427

## Python set-up

In [1]:
# system imports
import sys

import matplotlib.pyplot as plt

# analytic imports
import pandas as pd

# local imports
from abs_data_capture import (
    find_id,
    AbsLandingPage,
    get_abs_data,
    get_fs_constants,
    metacol,
    get_plot_constants,
)
from plotting import calc_growth, clear_chart_dir, plot_growth_finalise, set_chart_dir

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in the notebook
SHOW = False

## Get data from ABS

In [2]:
landing_page = AbsLandingPage(
    theme="economy",
    parent_topic="price-indexes-and-inflation",
    topic="producer-price-indexes-australia",
)
abs_dict = get_abs_data(landing_page)
source, CHART_DIR, Cat_ID, meta = get_fs_constants(abs_dict, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta)

## Plot

### Set-up some constants we will use across all plots

In [3]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Headline PPI for Australia

In [4]:
quarterly_phrase = "Percentage change from previous quarter"
annual_phrase = "Percentage change from corresponding quarter of previous year"
final = "Final ;  Total (Source)"

In [5]:
get = {
    (final, "1"): "Final Demand",
    ("Coal mining", "11"): "Coal mining",
    ("Manufacturing division", "12"): "Manufacturing",
    # ('Building construction Australia', '17'): 'Building construction Australia',
    # ('Road freight transport ', '21'): ' Road freight transport ',
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    identifiers = []
    for phrase in annual_phrase, quarterly_phrase:
        terms = {
            table: metacol.table,
            select_phrase: metacol.did,
            phrase: metacol.did,
        }
        ident, units = find_id(meta, terms, verbose=False)
        identifiers.append(ident)

    # plot
    title = f"Growth: {label} PPI"
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = (
            None if plot_from is None else pd.Period(plot_from, freq=data.index.freq)
        )
        plot_growth_finalise(
            data[identifiers[0]],
            data[identifiers[1]],
            title=title,
            from_=p_plot_from,
            tag=plot_tag,
            rfooter=f"{source} {table}",
            lfooter="Australia: Producer Price Index. Original series",
            show=SHOW,
        )

### Lesser PPIs - no ABS pre-calculated percentages

In [6]:
get = {
    ("30 Building construction Australia ;", "17"): "Building construction Australia",
    ("Road freight transport ", "21"): "Road freight transport",
    ("Employment services", "25"): "Employment services",
}

for key, label in get.items():
    select_phrase, table = key
    data = abs_dict[table]
    terms = {
        table: metacol.table,
        select_phrase: metacol.did,
        "Index Number": metacol.did,
    }
    ident, units = find_id(meta, terms, verbose=False)
    growth = calc_growth(abs_dict[table][ident])

    # plot
    title = f"Growth: {label} PPI"
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = (
            None if plot_from is None else pd.Period(plot_from, freq=data.index.freq)
        )
        plot_growth_finalise(
            *growth,
            title=title,
            from_=p_plot_from,
            tag=plot_tag,
            rfooter=f"{source} {table}",
            lfooter=f"Australia: Producer Price Index. Original series",
            show=SHOW,
        )

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri Apr 26 2024 14:43:12

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.22.2

sys       : 3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:35:20) [Clang 16.0.6 ]
matplotlib: 3.8.4
pandas    : 2.2.2

Watermark: 2.4.3



In [8]:
print("Finished")

Finished
